#Zadanie 4 (7 pkt)
Celem zadania jest zaimplementowanie algorytmu drzewa decyzyjnego ID3 dla zadania klasyfikacji. Trening i test należy przeprowadzić dla zbioru Iris. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania drzewa dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Implementacja funkcji entropii - **0.5 pkt**
* Implementacja funkcji entropii zbioru - **0.5 pkt**
* Implementacja funkcji information gain - **0.5 pkt**
* Zbudowanie poprawnie działającego drzewa klasyfikacyjnego i przetestowanie go na wspomnianym wcześniej zbiorze testowym. Jeśli w liściu występuje kilka różnych klas, decyzją jest klasa większościowa. Policzenie accuracy i wypisanie parami klasy rzeczywistej i predykcji. - **4 pkt**
* Przeprowadzenie eksperymentów dla różnych głębokości drzew i podziałów zbioru treningowego i testowego (zmiana wartości argumentu test_size oraz usunięcie random_state). W tym przypadku dla każdego eksperymentu należy wykonać kilka uruchomień programu i wypisać dla każdego uruchomienia accuracy. - **1.5 pkt**


In [17]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import itertools

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
def entropy_func(class_count, num_samples):
    if class_count == 0:
        return 0
    p = class_count / num_samples
    return -(p * np.log(p))


class Group:
    def __init__(self, group_classes):
        self.group_classes = group_classes
        self.entropy = self.group_entropy()

    def __len__(self):
        return len(self.group_classes)

    def group_entropy(self):
        count_samples = Counter(self.group_classes)
        num_samples = len(self)

        sum = 0
        for _, amt in count_samples.items():
            sum += entropy_func(amt, num_samples)

        return sum


class Node:
    def __init__(self, split_feature, split_val, depth=None, child_node_a=None, child_node_b=None, val=None):
        self.split_feature = split_feature # Index of the attribute in the data array
        self.split_val = split_val
        self.depth = depth
        self.child_node_a = child_node_a # Left branch
        self.child_node_b = child_node_b # Right branch
        self.val = val

    def predict(self, data):
        if self.val is not None:
            return self.val
        return self.child_node_a.predict(data)\
    if data[self.split_feature] < self.split_val\
    else self.child_node_b.predict(data)


class DecisionTreeClassifier():
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
        self.tree = None

    @staticmethod
    def get_split_entropy(group_a: Group, group_b: Group):
        size = len(group_a) + len(group_b)
        return (len(group_a) / size) * group_a.entropy + (len(group_b) / size) * group_b.entropy

    def get_information_gain(self, parent_group: Group, child_group_a: Group, child_group_b: Group):
        return parent_group.entropy - DecisionTreeClassifier.get_split_entropy(child_group_a, child_group_b)

    def get_best_feature_split(self, feature_values, classes):
        best_val = 0
        max_inf_gain = float("-inf")
        for val in set(feature_values):
            group_a = Group([classes[i] for i in range(len(classes)) if val > feature_values[i]])
            group_b = Group([classes[i] for i in range(len(classes)) if val <= feature_values[i]])
            parent_group = Group(classes)

            inf_gain = self.get_information_gain(parent_group, group_a, group_b)
            if inf_gain > max_inf_gain:
                best_val = val
                max_inf_gain = inf_gain

        return best_val, max_inf_gain

    def get_best_split(self, data, classes):
        max_inf_gain = float("-inf")
        split_feature = 0
        split_val = 0
        for i in range(len(data[0])):
            column = data[:, i]
            val, inf_gain = self.get_best_feature_split(column, classes)
            if inf_gain > max_inf_gain:
                split_feature = i
                split_val = val
                max_inf_gain = inf_gain

        return split_feature, split_val

    def build_tree(self, data, classes, depth=0):
        split_feature, split_val = self.get_best_split(np.array(data), classes)

        child_a_indexes = data[:, split_feature] < split_val
        child_b_indexes = data[:, split_feature] >= split_val

        child_a_data = data[child_a_indexes]
        child_a_classes = classes[child_a_indexes]
        child_b_data = data[child_b_indexes]
        child_b_classes = classes[child_b_indexes]
        if (len(child_a_data) == 0 or len(child_b_data) == 0) and depth == 0:
            most_common_class = Counter(classes).most_common(1)[0][0]
            self.tree = Node(split_feature=None, split_val=None, val=most_common_class)

        # we have reached max depth or its no use going deeper cause one of the child without data will cause errors
        if depth >= self.max_depth or len(child_a_data) == 0 or len(child_b_data) == 0:
            most_common_class = Counter(classes).most_common(1)[0][0]
            return Node(split_feature=None, split_val=None, val=most_common_class)

        child_node_a = self.build_tree(child_a_data, child_a_classes, depth+1)
        child_node_b = self.build_tree(child_b_data, child_b_classes, depth+1)

        if depth == 0:
            # we have to assign root node
            self.tree = Node(split_feature, split_val, 0, child_node_a, child_node_b)
            return None
        else:
            return Node(split_feature, split_val, 0, child_node_a, child_node_b)


    def predict(self, data):
        return self.tree.predict(data)

In [ ]:
class Solution:
    def __init__(self):
        pass

    def run_predictions(self, x_train, y_train, x_test, y_test, max_depth=3, verbose=0, round_acc = None):
        dc = DecisionTreeClassifier(max_depth)
        dc.build_tree(x_train, y_train)
        correct, count = 0, 0
        for sample, gt in zip(x_test, y_test):

            prediction = dc.predict(sample)
            if verbose > 0:
                print(f'Predicted: {prediction};\tCorrect prediciton: {gt}')
            count += 1
            correct += 1 if prediction == gt else 0

        accuracy = correct/count
        if verbose > 0:
            print(f'Accuracy: {accuracy}')
        return round(accuracy, round_acc) if round_acc else accuracy

    def simulate_depths(self, depths, x_test, y_test):
        y_axis = [self.run_predictions(x_train, y_train, x_test, y_test, depth, 0, 2) for depth in depths]
        self._plot_bar(depths, y_axis, "depths", "accuracy", "Change of accurece depth")

    def simulate_training_set(self, train_sizes, depth):
        x_train_set, y_train_set = self._get_set([0, 2], train_sizes, [None])

        y_axis = [self.run_predictions(x_train, y_train, x_test, y_test, depth, 0, 2) for x_train, y_train in zip(x_train_set, y_train_set)]
        self._plot_bar(train_sizes, y_axis, "train_size", "accuracy", f"Change of accuracy due to training size, depth = {depth}")

    def simulate_test_set(self , test_sizes, depth, train_size = None):
        y_axis = []
        y_average = []
        for i in range(100):
            x_test_set, y_test_set = self._get_set([1, 3], [train_size], test_sizes)
            y_axis.append([self.run_predictions(x_train, y_train, x_test, y_test, depth, 0, 2) for x_test, y_test in zip(x_test_set, y_test_set)])

        for i in range(len(test_sizes)):
            y_average[i] = np.average(y_axis, i)
            
        print(y_average)
        self._plot_bar(test_sizes, y_average, "test_size", "accuracy", f"Change of accuracy due to training size, depth = {depth}, train_size = {train_size}")

    def simulate_train_test_set(self, train_sizes, test_sizes, depth):
        for element in train_sizes:
            self.simulate_test_set(test_sizes, depth, train_size=element)

    """
    ids - list of attributes you want
    0 - x_train
    1 - x_test
    2 - y_train
    3 - y_test
    """
    def _get_set(self, ids, train_sizes=None, test_sizes=None):

        result_set = {}
        for id in ids:
            result_set[id] = []

        for train_size in train_sizes:
            for test_size in test_sizes:
                data = train_test_split(x, y, train_size=train_size, test_size=test_size)
                for id in ids:
                    result_set[id].append(data[id])

        return tuple(result_set.values())



    def _plot_bar(self, x_axis, y_axis, x_label, y_label, title):
        bar_width = 0.55
        x = np.arange(len(x_axis))

        plt.figure(figsize=(10, 6))
        bars = plt.bar(x, y_axis, width=bar_width, color='blue')

        for bar in bars:
            height = bar.get_height()
            plt.text( bar.get_x() + bar.get_width()/2, height, f'{height}',
                    ha='center', va='bottom')

        plt.xlabel(x_label)
        plt.ylabel(y_label)
        plt.title(title)
        # plt.tick_params(axis='x', labelrotation=90)
        plt.xticks(x, x_axis)  # Set x-axis tick labels to category names
        plt.tight_layout()         # Adjust layout for better fit

        plt.show()

In [ ]:
s = Solution()
# s.simulate_depths(range(1, 9), x_test, y_test)
# s.simulate_training_set([1, 5, 10, 25, 50, 100], 3)
s.simulate_test_set([1, 5, 10, 25, 50, 100], 3)
# s.simulate_train_test_set([1, 5, 10, 25, 50], [1, 5, 10, 25, 50], 3)
# dc.build_tree(x_train, y_train)
# count = 0
# correct = 0
# for sample, gt in zip(x_test, y_test):
#     count += 1
#     prediction = dc.predict(sample)
#     correct += 1 if prediction == gt else 0
# print(correct, count)


NameError: name 'Solution' is not defined